# Predictive Maintenance Workflow using PraisonAIAgents

This notebook demonstrates how to build a predictive maintenance workflow using multiple AI agents.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/Predictive_Maintenance_Multi-Agent_Workflow.ipynb)

# Dependencies

In [ ]:
!pip install praisonaiagents

# Set your OpenAI API key

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'enter your api key'

# Import required modules

In [ ]:
from praisonaiagents import Agent, Task, PraisonAIAgents
import time
from typing import Dict, List
import asyncio

# Define helper functions

In [ ]:

def collect_sensor_data():
    return {
        "temperature": 75 + (int(time.time()) % 20),
        "vibration": 0.5 + (int(time.time()) % 10) / 10,
        "pressure": 100 + (int(time.time()) % 50),
        "noise_level": 60 + (int(time.time()) % 30)
    }

def analyze_performance():
    return {
        "efficiency": 0.8 + (int(time.time()) % 20) / 100,
        "uptime": 0.95 + (int(time.time()) % 5) / 100,
        "output_quality": 0.9 + (int(time.time()) % 10) / 100
    }

def detect_anomalies(sensor_data: Dict, performance: Dict):
    anomalies = []
    if sensor_data["temperature"] > 90:
        anomalies.append({"type": "temperature_high", "severity": "critical"})
    if sensor_data["vibration"] > 1.2:
        anomalies.append({"type": "vibration_excess", "severity": "warning"})
    if performance["efficiency"] < 0.85:
        anomalies.append({"type": "efficiency_low", "severity": "warning"})
    return anomalies

def predict_failures(anomalies: List[Dict]):
    predictions = []
    severity_scores = {"critical": 0.9, "warning": 0.6}
    for anomaly in anomalies:
        predictions.append({
            "component": anomaly["type"].split("_")[0],
            "probability": severity_scores[anomaly["severity"]],
            "timeframe": "24_hours" if anomaly["severity"] == "critical" else "7_days"
        })
    return predictions

def schedule_maintenance(predictions: List[Dict]):
    schedule = []
    for pred in predictions:
        schedule.append({
            "component": pred["component"],
            "priority": "immediate" if pred["timeframe"] == "24_hours" else "planned",
            "estimated_duration": "2_hours",
            "required_parts": ["replacement_" + pred["component"]]
        })
    return schedule

# Define agents and tasks

In [ ]:

sensor_monitor = Agent("Sensor Monitor", "Data Collection", "Collect sensor data", "Monitor and collect sensor readings", [collect_sensor_data])
performance_analyzer = Agent("Performance Analyzer", "Performance Analysis", "Analyze equipment performance", "Analyze operational metrics", [analyze_performance])
anomaly_detector = Agent("Anomaly Detector", "Anomaly Detection", "Detect operational anomalies", "Identify abnormal patterns", [detect_anomalies])
failure_predictor = Agent("Failure Predictor", "Failure Prediction", "Predict potential failures", "Predict equipment failures", [predict_failures])
maintenance_scheduler = Agent("Maintenance Scheduler", "Maintenance Planning", "Schedule maintenance activities", "Plan and schedule maintenance", [schedule_maintenance])

sensor_task = Task("collect_data", "Collect sensor data", "Sensor readings", sensor_monitor, is_start=True, next_tasks=["analyze_performance"], async_execution=True)
performance_task = Task("analyze_performance", "Analyze performance metrics", "Performance analysis", performance_analyzer, next_tasks=["detect_anomalies"], async_execution=True)
anomaly_task = Task("detect_anomalies", "Detect operational anomalies", "Detected anomalies", anomaly_detector, next_tasks=["predict_failures"], context=[sensor_task, performance_task])
prediction_task = Task("predict_failures", "Predict potential failures", "Failure predictions", failure_predictor, next_tasks=["schedule_maintenance"], task_type="decision", condition={"critical": ["schedule_maintenance"], "warning": ["schedule_maintenance"], "normal": ""})
scheduling_task = Task("schedule_maintenance", "Schedule maintenance activities", "Maintenance schedule", maintenance_scheduler, context=[prediction_task])

# Create and run the workflow

In [ ]:

workflow = PraisonAIAgents(
    agents=[sensor_monitor, performance_analyzer, anomaly_detector, failure_predictor, maintenance_scheduler],
    tasks=[sensor_task, performance_task, anomaly_task, prediction_task, scheduling_task],
    process="workflow",
    verbose=True
)

async def main():
    print("\nStarting Predictive Maintenance Workflow...")
    print("=" * 50)
    results = await workflow.astart()
    print("\nMaintenance Planning Results:")
    print("=" * 50)
    for task_id, result in results["task_results"].items():
        if result:
            print(f"\nTask: {task_id}\nResult: {result.raw}\n{'-'*50}")

await main()

In [4]:
print("""
[Starting Predictive Maintenance Workflow...
==================================================
╭─ Agent Info ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                              │
│  👤 Agent: Sensor Monitor                                                                                                                    │
│  Role: Data Collection                                                                                                                       │
│  Tools: collect_sensor_data                                                                                                                  │
│                                                                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Agent Info ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                              │
│  👤 Agent: Performance Analyzer                                                                                                              │
│  Role: Performance Analysis                                                                                                                  │
│  Tools: analyze_performance                                                                                                                  │
│                                                                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[20:01:26] INFO     [20:01:26] process.py:429 INFO Task schedule_maintenance has no next tasks, ending workflow                   process.py:429

Maintenance Planning Results:
==================================================

Task: 0
Result: The sensor readings you have collected are as follows:

- **Temperature**: 86°F
- **Vibration**: 0.6 (units not specified, but typically measured in g-forces or mm/s)
- **Pressure**: 101 (units not specified, but typically measured in kPa or psi)
- **Noise Level**: 81 dB

Here's a brief analysis of these readings:

1. **Temperature**: At 86°F, the temperature is relatively warm. Depending on the context (e.g., industrial equipment, environmental monitoring), this could be within normal operating conditions or might require cooling measures if it's above the optimal range.

2. **Vibration**: A vibration level of 0.6 is generally low, but the significance depends on the type of equipment being monitored. For precision machinery, even small vibrations can be critical, whereas for more robust equipment, this might be negligible.

3. **Pressure**: A pressure reading of 101 is often within normal ranges for many systems, but without specific units or context, it's hard to determine if this is optimal or requires adjustment.

4. **Noise Level**: At 81 dB, the noise level is relatively high. Prolonged exposure to noise levels above 85 dB can be harmful to hearing, so if this is a workplace environment, it might be necessary to implement noise reduction measures or provide hearing protection.

Overall, these readings should be compared against the specific operational thresholds and safety standards relevant to the equipment or environment being monitored. If any values are outside of acceptable ranges, further investigation or corrective actions may be needed.
--------------------------------------------------

Task: 1
Result: Based on the provided operational metrics, here's an analysis of the equipment performance:

1. **Efficiency (94%)**:
   - The equipment is operating at a high efficiency level, with 94% of the input being effectively converted into useful output. This suggests
that the equipment is well-maintained and optimized for performance. However, there is still a 6% margin for improvement, which could be addressed by identifying and minimizing any inefficiencies in the process.

2. **Uptime (99%)**:
   - The equipment has an excellent uptime rate of 99%, indicating that it is available and operational almost all the time. This is a strong indicator of reliability and suggests that downtime due to maintenance or unexpected failures is minimal. Maintaining this level of uptime should
be a priority, as it directly impacts productivity and operational continuity.

3. **Output Quality (94%)**:
   - The output quality is also at 94%, which is a positive sign that the equipment is producing high-quality products or results. However, similar to efficiency, there is room for improvement. Efforts could be made to identify any factors that might be affecting quality, such as calibration issues, material inconsistencies, or process deviations.

**Overall Assessment**:
The equipment is performing well across all key metrics, with high efficiency, uptime, and output quality. To further enhance performance, focus should be placed on fine-tuning processes to close the small gaps in efficiency and quality. Regular maintenance, monitoring, and process optimization can help sustain and potentially improve these metrics.
--------------------------------------------------]
""")


[Starting Predictive Maintenance Workflow...
╭─ Agent Info ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                              │
│  👤 Agent: Sensor Monitor                                                                                                                    │
│  Role: Data Collection                                                                                                                       │
│  Tools: collect_sensor_data                                                                                                                  │
│                                                                                                                                              │
╰────────────────────────────────────────────────────────────────────────────────────